### Web-Scraping images

In [7]:
import requests
import json
import os

# Define the API endpoint
url = "https://pastvu.com/api2"

# Define the bounding box for Moldova using its extreme points
geometry = {
    "type": "Polygon",
    "coordinates": [
        [
            [26.616475, 45.466549],  # SW
            [30.163753, 45.466549],  # SE
            [30.163753, 48.491593],  # NE
            [26.616475, 48.491593],  # NW
            [26.616475, 45.466549]   # Close the polygon
        ]
    ]
}

# Define API parameters
params = {
    "method": "photo.getByBounds",
    "params": json.dumps({
        "z": 16,  # High zoom level for individual photos
        "geometry": geometry
    })
}

# Send the API request
response = requests.get(url, params=params)
print(response.json())

# Parse the response
response_json = response.json()  # Extract JSON data from the response
photos = response_json.get("result", {}).get("photos", [])

if photos:
    print(f"Found {len(photos)} photos.")

    # Create a folder for photos
    os.makedirs("photos_moldova", exist_ok=True)

    # Download photos
    for photo in photos:
        photo_url = f"https://pastvu.com/_p/d/{photo['file']}"  # Use "_p/a/" for original size
        photo_path = os.path.join("photos_moldova", f"{photo['cid']}.jpg")
        
        # Print photo metadata
        print(f"Photo ID: {photo['cid']}, Title: {photo['title']}, Year: {photo['year']}")
        print(f"Geo: {photo['geo']}, URL: {photo_url}")
        
        # Download the photo
        photo_response = requests.get(photo_url)
        if photo_response.status_code == 200:
            with open(photo_path, "wb") as f:
                f.write(photo_response.content)
            print(f"Saved photo: {photo_path}")
        else:
            print(f"Failed to download photo: {photo_url}")
else:
    print("No photos found.")

{'result': {'z': 16, 'photos': [{'cid': 1716566, 'file': 'x/z/5/xz52ohlundqap8ligy.jpg', 'title': 'Mărăşeşti. Aspect din timpul vizitei conducătorilor de partid şi de stat în regiunea Galaţi', 'dir': 'w', 'geo': [45.872381, 27.209433], 'year': 1966, 'year2': 1966}, {'cid': 2054801, 'file': 'h/c/t/hcty5id94ysrb5hccl.jpg', 'title': 'Combinatului Chimic Gheorghe Gheorghiu-Dej', 'dir': 'e', 'geo': [46.261959, 26.791684], 'year': 1970, 'year2': 1970}, {'cid': 397337, 'file': 't/c/z/tcztzj45ccmyfh6bu4.jpg', 'title': 'Onești', 'year': 1975, 'year2': 1975, 'dir': 'ne', 'geo': [46.244102, 26.762217]}, {'cid': 1079085, 'file': 'w/g/6/wg6ahufh2toqmph5cx.jpg', 'title': 'Onești, Aleea Parcului', 'year': 1965, 'year2': 1965, 'dir': 'w', 'geo': [46.252113, 26.765694]}, {'cid': 2054800, 'file': 'v/m/5/vm5zn4zth88631yx3n.jpg', 'title': 'Combinatul de cauciuc sintetic și produse petrochimice Onești', 'dir': 'n', 'geo': [46.263286, 26.801158], 'year': 1960, 'year2': 1965}, {'cid': 714114, 'file': 'd/z/5/

### Screen grabbing from Archived videos

In [9]:
import cv2
import os

def extract_frames_from_videos(input_dir, output_dir, interval=5):
    """
    Extract frames from all videos in a directory at regular intervals.

    Args:
        input_dir (str): Path to the folder containing videos.
        output_dir (str): Path to save the extracted screenshots.
        interval (int): Time interval (in seconds) between screenshots.
    """
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)

    # List all video files in the input directory
    video_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))]

    if not video_files:
        print("No video files found in the specified directory.")
        return

    for video_file in video_files:
        video_path = os.path.join(input_dir, video_file)
        video_output_dir = os.path.join(output_dir, os.path.splitext(video_file)[0])
        os.makedirs(video_output_dir, exist_ok=True)

        # Process each video
        print(f"Processing video: {video_file}")
        extract_frames(video_path, video_output_dir, interval)

def extract_frames(video_path, output_dir, interval=5):
    """
    Extract frames from a single video at regular intervals.

    Args:
        video_path (str): Path to the video file.
        output_dir (str): Directory to save screenshots.
        interval (int): Time interval (in seconds) between screenshots.
    """
    video = cv2.VideoCapture(video_path)
    fps = int(video.get(cv2.CAP_PROP_FPS))  # Frames per second
    frame_interval = interval * fps
    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))

    video_basename = os.path.splitext(os.path.basename(video_path))[0]
    count = 0  # Frame counter
    screenshot_index = 0

    while True:
        ret, frame = video.read()
        if not ret:
            break  # End of video

        if count % frame_interval == 0:
            filename = os.path.join(output_dir, f"{video_basename}_screenshot_{screenshot_index:03d}.png")
            cv2.imwrite(filename, frame)
            screenshot_index += 1

        count += 1

    video.release()
    print(f"Screenshots saved to {output_dir}")

# Usage
input_directory = "arhiva_gov_md_video"
output_directory = "arhiva_gov_md_video_screenshots"
interval_seconds = 5  # Set the interval in seconds

extract_frames_from_videos(input_directory, output_directory, interval_seconds)

Processing video: 3165454909733905407_27643612181.mp4
Screenshots saved to arhiva_gov_md_video_screenshots\3165454909733905407_27643612181
Processing video: 3257653653845557037_44257248903.mp4
Screenshots saved to arhiva_gov_md_video_screenshots\3257653653845557037_44257248903
Processing video: 3282444846739282713_44257248903.mp4
Screenshots saved to arhiva_gov_md_video_screenshots\3282444846739282713_44257248903
Processing video: 3283013159060168499_44257248903.mp4
Screenshots saved to arhiva_gov_md_video_screenshots\3283013159060168499_44257248903
Processing video: 3285902569971725062_44257248903.mp4
Screenshots saved to arhiva_gov_md_video_screenshots\3285902569971725062_44257248903
Processing video: 3293218419896743745_44257248903.mp4
Screenshots saved to arhiva_gov_md_video_screenshots\3293218419896743745_44257248903
Processing video: 3297458896321884292_44257248903.mp4
Screenshots saved to arhiva_gov_md_video_screenshots\3297458896321884292_44257248903
Processing video: 329825747